# Experiment 054: Simpler Model with Stronger Regularization

**Hypothesis**: The CV-LB relationship (LB = 4.31*CV + 0.0525) has an intercept > target. We need to CHANGE this relationship, not just improve CV. Stronger regularization may help.

**Key changes from baseline (exp_030):**
1. Use Spange + Arrhenius ONLY (drop DRFP - it consistently hurts)
2. Higher GP weight in ensemble (0.4 vs 0.15)
3. Stronger regularization in MLP (dropout 0.4, weight_decay 1e-3)
4. Simpler MLP architecture ([64, 32] vs [128, 64])

**Why this should work:**
- Simpler features may generalize better to unseen solvents
- Stronger regularization prevents overfitting to CV
- Higher GP weight provides uncertainty-aware predictions
- This may change the CV-LB relationship

In [1]:
import sys
sys.path.insert(0, '/home/code/experiments/049_manual_ood_handling')

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# Load data
from utils_local import load_data, load_features, generate_leave_one_out_splits, generate_leave_one_ramp_out_splits

print("Loading data...")
X_single_raw, Y_single = load_data("single_solvent")
X_full_raw, Y_full = load_data("full")

print(f"Single solvent: {X_single_raw.shape}, Mixtures: {X_full_raw.shape}")

# Load features
spange = load_features("spange_descriptors")
print(f"Spange: {spange.shape}")

Loading data...
Single solvent: (656, 3), Mixtures: (1227, 5)
Spange: (26, 13)


In [2]:
# Prepare datasets with Spange features ONLY (no DRFP)
def prepare_single_solvent_dataset(X_raw, spange):
    """Prepare single solvent dataset with Spange features only"""
    solvent_name = X_raw['SOLVENT NAME'].values
    spange_features = spange.loc[solvent_name].values
    time = X_raw['Residence Time'].values
    temp = X_raw['Temperature'].values
    
    spange_cols = spange.columns.tolist()
    df = pd.DataFrame(spange_features, columns=spange_cols)
    df['TEMPERATURE'] = temp
    df['TIME'] = time
    df['SOLVENT NAME'] = solvent_name
    
    return df

def prepare_mixture_dataset(X_raw, spange):
    """Prepare mixture dataset with Spange features only"""
    solvent_a = X_raw['SOLVENT A NAME'].values
    solvent_b = X_raw['SOLVENT B NAME'].values
    solvent_b_pct = X_raw['SolventB%'].values / 100.0
    
    spange_a = spange.loc[solvent_a].values
    spange_b = spange.loc[solvent_b].values
    spange_mix = (1 - solvent_b_pct[:, None]) * spange_a + solvent_b_pct[:, None] * spange_b
    
    solvent_name = [f"{a}.{b}" for a, b in zip(solvent_a, solvent_b)]
    time = X_raw['Residence Time'].values
    temp = X_raw['Temperature'].values
    
    spange_cols = spange.columns.tolist()
    df = pd.DataFrame(spange_mix, columns=spange_cols)
    df['TEMPERATURE'] = temp
    df['TIME'] = time
    df['SOLVENT NAME'] = solvent_name
    df['SOLVENT A NAME'] = solvent_a
    df['SOLVENT B NAME'] = solvent_b
    df['SolventB%'] = X_raw['SolventB%'].values
    
    return df

X_single = prepare_single_solvent_dataset(X_single_raw, spange)
X_mix = prepare_mixture_dataset(X_full_raw, spange)

print(f"Single solvent dataset: {X_single.shape}")
print(f"Mixture dataset: {X_mix.shape}")

Single solvent dataset: (656, 16)
Mixture dataset: (1227, 19)


In [3]:
# Feature extraction with Arrhenius kinetics
def get_spange_features(X_data):
    spange_cols = ['dielectric constant', 'ET(30)', 'alpha', 'beta', 'pi*', 
                   'SA', 'SB', 'SP', 'SdP', 'N', 'n', 'f(n)', 'delta']
    return X_data[spange_cols].values

def get_arrhenius_features(X_data):
    T = X_data['TEMPERATURE'].values
    t = X_data['TIME'].values
    T_kelvin = T + 273.15
    inv_T = 1.0 / T_kelvin
    ln_t = np.log(t + 1e-6)
    interaction = inv_T * ln_t
    return np.column_stack([inv_T, ln_t, interaction, T, t])

def prepare_features(X_data):
    """Prepare features (Spange + Arrhenius only, NO DRFP)"""
    spange = get_spange_features(X_data)
    arrhenius = get_arrhenius_features(X_data)
    return np.hstack([spange, arrhenius])

print("Feature extraction functions defined")
print(f"Feature dimension: 13 (Spange) + 5 (Arrhenius) = 18")

Feature extraction functions defined
Feature dimension: 13 (Spange) + 5 (Arrhenius) = 18


In [4]:
# Simpler MLP Model with stronger regularization
class SimplerMLP(nn.Module):
    def __init__(self, input_dim, hidden_dims=[64, 32], dropout=0.4):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(dropout)  # Higher dropout for regularization
            ])
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 3))
        layers.append(nn.Sigmoid())
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def train_simpler_mlp(X_train, Y_train, input_dim, epochs=200, lr=5e-4, 
                      weight_decay=1e-3, hidden_dims=[64, 32], dropout=0.4):
    """Train simpler MLP with stronger regularization"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SimplerMLP(input_dim, hidden_dims=hidden_dims, dropout=dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=20)
    criterion = nn.HuberLoss()
    
    X_tensor = torch.FloatTensor(X_train).to(device)
    Y_tensor = torch.FloatTensor(Y_train).to(device)
    
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        pred = model(X_tensor)
        loss = criterion(pred, Y_tensor)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
    
    return model

print("Simpler MLP model defined")

Simpler MLP model defined


In [5]:
# Simpler Regularized Model
class SimplerRegularizedModel:
    """
    Simpler model with stronger regularization:
    - Spange + Arrhenius features only (no DRFP)
    - Higher GP weight (0.4 vs 0.15)
    - Simpler MLP ([64, 32] vs [128, 64])
    - Stronger regularization (dropout 0.4, weight_decay 1e-3)
    """
    def __init__(self, gp_weight=0.4, mlp_weight=0.35, lgbm_weight=0.25):
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        
        self.scaler = StandardScaler()
        self.gp_models = []
        self.mlp_models = []
        self.lgbm_models = []
        
        self.input_dim = None
    
    def fit(self, X_train, Y_train):
        """Train all models"""
        # Prepare features (Spange + Arrhenius only)
        X_features = prepare_features(X_train)
        self.input_dim = X_features.shape[1]
        X_scaled = self.scaler.fit_transform(X_features)
        
        Y_values = Y_train.values
        
        # Train GP (on subset for speed, but larger subset for better uncertainty)
        n_gp = min(300, len(X_scaled))  # Larger subset for GP
        idx_gp = np.random.choice(len(X_scaled), n_gp, replace=False)
        for i in range(3):
            kernel = Matern(nu=2.5, length_scale=1.0) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(kernel=kernel, alpha=0.1, n_restarts_optimizer=3)
            gp.fit(X_scaled[idx_gp], Y_values[idx_gp, i])
            self.gp_models.append(gp)
        
        # Train MLP (3 models for bagging, simpler architecture)
        for _ in range(3):
            mlp = train_simpler_mlp(X_scaled, Y_values, self.input_dim, 
                                    epochs=200, hidden_dims=[64, 32], 
                                    dropout=0.4, weight_decay=1e-3)
            self.mlp_models.append(mlp)
        
        # Train LightGBM (with stronger regularization)
        lgbm_params = {
            'objective': 'regression',
            'metric': 'mse',
            'learning_rate': 0.02,  # Lower learning rate
            'max_depth': 4,  # Shallower trees
            'num_leaves': 15,  # Fewer leaves
            'reg_alpha': 0.5,  # Stronger L1 regularization
            'reg_lambda': 0.5,  # Stronger L2 regularization
            'verbose': -1,
            'n_estimators': 300
        }
        for i in range(3):
            model = lgb.LGBMRegressor(**lgbm_params)
            model.fit(X_scaled, Y_values[:, i])
            self.lgbm_models.append(model)
        
        return self
    
    def predict(self, X_test):
        """Predict using ensemble"""
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        X_features = prepare_features(X_test)
        X_scaled = self.scaler.transform(X_features)
        
        # GP predictions
        gp_preds = np.zeros((len(X_test), 3))
        for i, gp in enumerate(self.gp_models):
            gp_preds[:, i] = gp.predict(X_scaled)
        gp_preds = np.clip(gp_preds, 0, 1)
        
        # MLP predictions
        mlp_preds = []
        X_tensor = torch.FloatTensor(X_scaled).to(device)
        for mlp in self.mlp_models:
            mlp.eval()
            with torch.no_grad():
                pred = mlp(X_tensor).cpu().numpy()
            mlp_preds.append(pred)
        mlp_pred = np.mean(mlp_preds, axis=0)
        
        # LightGBM predictions
        lgbm_pred = np.zeros((len(X_test), 3))
        for i, model in enumerate(self.lgbm_models):
            lgbm_pred[:, i] = model.predict(X_scaled)
        lgbm_pred = np.clip(lgbm_pred, 0, 1)
        
        # Ensemble with higher GP weight
        final_pred = self.gp_weight * gp_preds + self.mlp_weight * mlp_pred + self.lgbm_weight * lgbm_pred
        
        return np.clip(final_pred, 0, 1)

print("SimplerRegularizedModel defined")

SimplerRegularizedModel defined


In [6]:
# Run CV for single solvents
print("Running Single Solvent CV with Simpler Regularized Model...")
print("="*60)

splits = list(generate_leave_one_out_splits(X_single, Y_single))
print(f"Number of folds: {len(splits)}")

solvent_errors = {}
all_preds = []
all_true = []

for fold_idx, (train_idx, test_idx) in enumerate(splits):
    X_train = X_single.iloc[train_idx]
    Y_train = Y_single.iloc[train_idx]
    X_test = X_single.iloc[test_idx]
    Y_test = Y_single.iloc[test_idx]
    
    test_solvent = X_test['SOLVENT NAME'].iloc[0]
    
    # Train model
    model = SimplerRegularizedModel(gp_weight=0.4, mlp_weight=0.35, lgbm_weight=0.25)
    model.fit(X_train, Y_train)
    
    # Predict
    preds = model.predict(X_test)
    
    # Calculate MSE
    mse = np.mean((preds - Y_test.values) ** 2)
    solvent_errors[test_solvent] = mse
    
    all_preds.append(preds)
    all_true.append(Y_test.values)
    
    print(f"Fold {fold_idx+1:2d}: {test_solvent:45s} MSE = {mse:.6f}")

all_preds = np.vstack(all_preds)
all_true = np.vstack(all_true)
single_mse = np.mean((all_preds - all_true) ** 2)
single_std = np.std([solvent_errors[s] for s in solvent_errors])

print(f"\nSimpler Regularized Single Solvent CV MSE: {single_mse:.6f} +/- {single_std:.6f}")

Running Single Solvent CV with Simpler Regularized Model...
Number of folds: 24


Fold  1: 1,1,1,3,3,3-Hexafluoropropan-2-ol             MSE = 0.028182


Fold  2: 2,2,2-Trifluoroethanol                        MSE = 0.032450


Fold  3: 2-Methyltetrahydrofuran [2-MeTHF]             MSE = 0.003994


Fold  4: Acetonitrile                                  MSE = 0.009324


...

Fold 20: Methyl Propionate                             MSE = 0.005382


Fold 21: THF [Tetrahydrofuran]                         MSE = 0.003699


Fold 22: Water.2,2,2-Trifluoroethanol                  MSE = 0.005634


Fold 23: Water.Acetonitrile                            MSE = 0.010331


Fold 24: tert-Butanol [2-Methylpropan-2-ol]            MSE = 0.008535

Simpler Regularized Single Solvent CV MSE: 0.011802 +/- 0.008246


In [ ]:
# Run CV for mixtures
print("\n" + "="*60)
print("Running Mixture CV with Simpler Regularized Model...")
print("="*60)

mix_splits = list(generate_leave_one_ramp_out_splits(X_mix, Y_full))
print(f"Number of folds: {len(mix_splits)}")

mix_errors = {}
mix_preds = []
mix_true = []

for fold_idx, (train_idx, test_idx) in enumerate(mix_splits):
    X_train = X_mix.iloc[train_idx]
    Y_train = Y_full.iloc[train_idx]
    X_test = X_mix.iloc[test_idx]
    Y_test = Y_full.iloc[test_idx]
    
    test_mixture = X_test['SOLVENT NAME'].iloc[0]
    
    # Train model
    model = SimplerRegularizedModel(gp_weight=0.4, mlp_weight=0.35, lgbm_weight=0.25)
    model.fit(X_train, Y_train)
    
    # Predict
    preds = model.predict(X_test)
    
    # Calculate MSE
    mse = np.mean((preds - Y_test.values) ** 2)
    mix_errors[test_mixture] = mse
    
    mix_preds.append(preds)
    mix_true.append(Y_test.values)
    
    print(f"Fold {fold_idx+1:2d}: {test_mixture:55s} MSE = {mse:.6f}")

mix_preds = np.vstack(mix_preds)
mix_true = np.vstack(mix_true)
mix_mse = np.mean((mix_preds - mix_true) ** 2)
mix_std = np.std([mix_errors[s] for s in mix_errors])

print(f"\nSimpler Regularized Mixture CV MSE: {mix_mse:.6f} +/- {mix_std:.6f}")

In [ ]:
# Calculate overall CV score
print("\n" + "="*60)
print("Simpler Regularized Model Overall Results")
print("="*60)

n_single = len(all_true)
n_mix = len(mix_true)
n_total = n_single + n_mix

overall_mse = (n_single * single_mse + n_mix * mix_mse) / n_total

print(f"\nSingle Solvent CV MSE: {single_mse:.6f} +/- {single_std:.6f} (n={n_single})")
print(f"Mixture CV MSE: {mix_mse:.6f} +/- {mix_std:.6f} (n={n_mix})")
print(f"Overall CV MSE: {overall_mse:.6f}")

print(f"\nBaseline (exp_030): CV = 0.008298")
print(f"Improvement vs baseline: {(0.008298 - overall_mse) / 0.008298 * 100:.1f}%")

if overall_mse < 0.008298:
    print("\n✓ BETTER than baseline!")
else:
    print("\n✗ WORSE than baseline.")

In [ ]:
# Final Summary
print("\n" + "="*60)
print("EXPERIMENT 054 SUMMARY")
print("="*60)

print(f"\nSimpler Regularized Model:")
print(f"  Features: Spange + Arrhenius only (18 features, NO DRFP)")
print(f"  GP weight: 0.4 (vs 0.15 in baseline)")
print(f"  MLP: [64, 32] with dropout 0.4, weight_decay 1e-3")
print(f"  LGBM: max_depth 4, num_leaves 15, stronger regularization")
print(f"\n  Single Solvent CV: {single_mse:.6f}")
print(f"  Mixture CV: {mix_mse:.6f}")
print(f"  Overall CV: {overall_mse:.6f}")
print(f"  vs Baseline (exp_030): {(overall_mse - 0.008298) / 0.008298 * 100:+.1f}%")

print("\nKey insights:")
print("1. Simpler features (no DRFP) may generalize better")
print("2. Higher GP weight provides stronger regularization")
print("3. Simpler MLP architecture prevents overfitting")

if overall_mse < 0.008298:
    print("\nCONCLUSION: Simpler regularized model IMPROVES CV!")
    print("This may change the CV-LB relationship.")
else:
    print("\nCONCLUSION: Simpler regularized model does NOT improve CV.")
    print("Stronger regularization alone doesn't help.")

print(f"\nRemaining submissions: 5")
print(f"Best model: exp_030 (GP 0.15 + MLP 0.55 + LGBM 0.3) with CV 0.008298, LB 0.0877")